# Streamlit Image Analysis Application

> **Created by [Build Fast with AI](https://www.buildfastwithai.com)**

This notebook shows you how to build an interactive image analysis application using Streamlit and Gemini 3 Pro's vision capabilities.

## What you'll learn:
- Working with Gemini's vision API
- Building image upload interfaces
- Image preprocessing and handling
- Multiple analysis modes
- Batch image processing
- Creating interactive visualizations

## 1. Installation and Setup

In [ ]:
!pip install -q streamlit google-generativeai pillow opencv-python matplotlib

In [ ]:
import os
import google.generativeai as genai
from PIL import Image
import io
from IPython.display import display, Markdown

In [ ]:
# Configure API key
try:
    from google.colab import userdata
    GOOGLE_API_KEY = userdata.get('GOOGLE_API_KEY')
except:
    GOOGLE_API_KEY = os.environ.get('GOOGLE_API_KEY', 'your-api-key-here')

genai.configure(api_key=GOOGLE_API_KEY)

## 2. Basic Image Analysis with Gemini

In [ ]:
# Create a simple test image
from PIL import Image, ImageDraw, ImageFont
import numpy as np

def create_test_image():
    """Create a test image with shapes and text."""
    img = Image.new('RGB', (400, 300), color='white')
    draw = ImageDraw.Draw(img)
    
    # Draw shapes
    draw.rectangle([50, 50, 150, 150], fill='red', outline='black')
    draw.ellipse([200, 50, 300, 150], fill='blue', outline='black')
    draw.polygon([100, 200, 150, 250, 50, 250], fill='green', outline='black')
    
    # Add text
    try:
        font = ImageFont.truetype("/usr/share/fonts/truetype/dejavu/DejaVuSans-Bold.ttf", 20)
    except:
        font = ImageFont.load_default()
    
    draw.text((150, 280), "Test Image", fill='black', font=font)
    
    return img

test_image = create_test_image()
test_image.save('test_image.png')
display(test_image)

In [ ]:
# Analyze the image
model = genai.GenerativeModel('gemini-3-pro')

# Load and analyze image
img = Image.open('test_image.png')

response = model.generate_content([
    "Describe this image in detail. What shapes and colors do you see?",
    img
])

print("Image Analysis:")
print("=" * 80)
display(Markdown(response.text))

## 3. Different Analysis Modes

In [ ]:
class ImageAnalyzer:
    """Analyze images with different modes."""
    
    def __init__(self):
        self.model = genai.GenerativeModel('gemini-3-pro')
    
    def describe(self, image: Image.Image) -> str:
        """General description of the image."""
        response = self.model.generate_content([
            "Provide a detailed description of this image.",
            image
        ])
        return response.text
    
    def detect_objects(self, image: Image.Image) -> str:
        """Detect and list objects in the image."""
        response = self.model.generate_content([
            "List all objects you can identify in this image. Format as a bullet list.",
            image
        ])
        return response.text
    
    def extract_text(self, image: Image.Image) -> str:
        """Extract text from the image (OCR)."""
        response = self.model.generate_content([
            "Extract all text visible in this image. If there's no text, say 'No text found'.",
            image
        ])
        return response.text
    
    def analyze_colors(self, image: Image.Image) -> str:
        """Analyze dominant colors."""
        response = self.model.generate_content([
            "Identify the dominant colors in this image and describe the color scheme.",
            image
        ])
        return response.text
    
    def identify_style(self, image: Image.Image) -> str:
        """Identify artistic style or genre."""
        response = self.model.generate_content([
            "Describe the artistic style, mood, and visual characteristics of this image.",
            image
        ])
        return response.text
    
    def answer_question(self, image: Image.Image, question: str) -> str:
        """Answer specific questions about the image."""
        response = self.model.generate_content([question, image])
        return response.text

# Test different analysis modes
analyzer = ImageAnalyzer()
img = Image.open('test_image.png')

modes = {
    "Description": lambda: analyzer.describe(img),
    "Object Detection": lambda: analyzer.detect_objects(img),
    "Text Extraction": lambda: analyzer.extract_text(img),
    "Color Analysis": lambda: analyzer.analyze_colors(img),
    "Style Analysis": lambda: analyzer.identify_style(img)
}

for mode_name, mode_func in modes.items():
    print(f"\n{'='*80}")
    print(f"{mode_name}:")
    print(f"{'='*80}")
    result = mode_func()
    print(result)

## 4. Basic Streamlit Image Analysis App

In [ ]:
# Save this as image_analysis_basic.py
basic_app_code = '''
import streamlit as st
import google.generativeai as genai
from PIL import Image
import os

st.set_page_config(
    page_title="Image Analysis",
    page_icon="🖼️",
    layout="wide"
)

st.title("🖼️ AI Image Analyzer")
st.caption("Upload an image and let AI analyze it")

# API Key
api_key = os.environ.get('GOOGLE_API_KEY')
if not api_key:
    api_key = st.sidebar.text_input("Google API Key", type="password")

if api_key:
    genai.configure(api_key=api_key)
    
    # File upload
    uploaded_file = st.file_uploader(
        "Choose an image",
        type=["jpg", "jpeg", "png", "webp"]
    )
    
    if uploaded_file:
        # Display image
        image = Image.open(uploaded_file)
        
        col1, col2 = st.columns([1, 1])
        
        with col1:
            st.subheader("Original Image")
            st.image(image, use_column_width=True)
            
            # Image info
            st.info(f"Size: {image.size[0]} x {image.size[1]} pixels")
        
        with col2:
            st.subheader("Analysis")
            
            # Analysis mode
            mode = st.selectbox(
                "Select Analysis Mode",
                ["Describe Image", "Detect Objects", "Extract Text", "Analyze Colors", "Custom Question"]
            )
            
            if mode == "Custom Question":
                question = st.text_input("Ask about the image:")
            else:
                question = None
            
            if st.button("Analyze", type="primary"):
                with st.spinner("Analyzing..."):
                    model = genai.GenerativeModel('gemini-3-pro')
                    
                    prompts = {
                        "Describe Image": "Provide a detailed description of this image.",
                        "Detect Objects": "List all objects you can identify in this image.",
                        "Extract Text": "Extract all text from this image. Say 'No text found' if there's none.",
                        "Analyze Colors": "Describe the dominant colors and color scheme of this image.",
                        "Custom Question": question if question else "Describe this image."
                    }
                    
                    prompt = prompts[mode]
                    response = model.generate_content([prompt, image])
                    
                    st.success("Analysis Complete!")
                    st.markdown(response.text)
    else:
        st.info("👆 Upload an image to get started")
        
        # Example use cases
        st.markdown("""
        ### What you can do:
        - 📝 Get detailed image descriptions
        - 🔍 Detect objects and elements
        - 📄 Extract text (OCR)
        - 🎨 Analyze colors and styles
        - ❓ Ask custom questions about images
        """)
else:
    st.warning("Please enter your API key to start")
'''

with open('image_analysis_basic.py', 'w') as f:
    f.write(basic_app_code)

print("Basic image analysis app saved to image_analysis_basic.py")
print("\nTo run: streamlit run image_analysis_basic.py")

## 5. Advanced Image Analysis App with Multiple Features

In [ ]:
# Save this as image_analysis_advanced.py
advanced_app_code = '''
import streamlit as st
import google.generativeai as genai
from PIL import Image, ImageEnhance, ImageFilter
import os
import io

st.set_page_config(
    page_title="Advanced Image Analysis",
    page_icon="🎨",
    layout="wide"
)

# Custom CSS
st.markdown("""
<style>
    .main-header {
        font-size: 3rem;
        font-weight: bold;
        color: #1f77b4;
        text-align: center;
        margin-bottom: 1rem;
    }
    .stButton>button {
        width: 100%;
    }
</style>
""", unsafe_allow_html=True)

st.markdown('<p class="main-header">🎨 Advanced AI Image Analyzer</p>', unsafe_allow_html=True)

# Sidebar
with st.sidebar:
    st.header("⚙️ Settings")
    
    api_key = os.environ.get('GOOGLE_API_KEY')
    if not api_key:
        api_key = st.text_input("Google API Key", type="password")
    
    st.divider()
    
    st.subheader("Image Preprocessing")
    enable_preprocessing = st.checkbox("Enable preprocessing", value=False)
    
    if enable_preprocessing:
        brightness = st.slider("Brightness", 0.5, 2.0, 1.0, 0.1)
        contrast = st.slider("Contrast", 0.5, 2.0, 1.0, 0.1)
        sharpness = st.slider("Sharpness", 0.5, 2.0, 1.0, 0.1)
    
    st.divider()
    
    st.subheader("Analysis Options")
    detail_level = st.select_slider(
        "Detail Level",
        options=["Brief", "Normal", "Detailed"],
        value="Normal"
    )

def preprocess_image(image, brightness, contrast, sharpness):
    """Apply image preprocessing."""
    # Brightness
    enhancer = ImageEnhance.Brightness(image)
    image = enhancer.enhance(brightness)
    
    # Contrast
    enhancer = ImageEnhance.Contrast(image)
    image = enhancer.enhance(contrast)
    
    # Sharpness
    enhancer = ImageEnhance.Sharpness(image)
    image = enhancer.enhance(sharpness)
    
    return image

if api_key:
    genai.configure(api_key=api_key)
    
    # Main content
    tab1, tab2, tab3 = st.tabs(["📤 Single Image", "📦 Batch Analysis", "🔄 Compare Images"])
    
    # Tab 1: Single Image Analysis
    with tab1:
        uploaded_file = st.file_uploader(
            "Upload an image",
            type=["jpg", "jpeg", "png", "webp"],
            key="single"
        )
        
        if uploaded_file:
            image = Image.open(uploaded_file)
            
            # Apply preprocessing if enabled
            if enable_preprocessing:
                processed_image = preprocess_image(image, brightness, contrast, sharpness)
            else:
                processed_image = image
            
            # Display images
            col1, col2 = st.columns(2)
            
            with col1:
                st.subheader("Image")
                st.image(processed_image, use_column_width=True)
                
                st.metric("Size", f"{image.size[0]} x {image.size[1]}")
                st.metric("Format", image.format)
            
            with col2:
                st.subheader("Analysis")
                
                # Analysis buttons
                analysis_types = [
                    ("🔍 Describe", "Provide a {} description of this image."),
                    ("📦 Objects", "List all objects in this image."),
                    ("📝 Text (OCR)", "Extract all text from this image."),
                    ("🎨 Colors", "Analyze the color palette and scheme."),
                    ("🖼️ Style", "Describe the artistic style and mood."),
                    ("🏷️ Tags", "Generate relevant tags for this image.")
                ]
                
                col_a, col_b, col_c = st.columns(3)
                columns = [col_a, col_b, col_c]
                
                for idx, (label, prompt_template) in enumerate(analysis_types):
                    with columns[idx % 3]:
                        if st.button(label, key=f"btn_{idx}"):
                            with st.spinner("Analyzing..."):
                                model = genai.GenerativeModel('gemini-3-pro')
                                
                                detail_map = {
                                    "Brief": "brief",
                                    "Normal": "detailed",
                                    "Detailed": "very detailed"
                                }
                                
                                prompt = prompt_template.format(detail_map[detail_level])
                                response = model.generate_content([prompt, processed_image])
                                
                                st.session_state.last_result = response.text
                
                # Custom question
                st.divider()
                custom_q = st.text_input("Ask a custom question:")
                if st.button("❓ Ask", key="custom_btn"):
                    if custom_q:
                        with st.spinner("Thinking..."):
                            model = genai.GenerativeModel('gemini-3-pro')
                            response = model.generate_content([custom_q, processed_image])
                            st.session_state.last_result = response.text
                
                # Display result
                if "last_result" in st.session_state:
                    st.divider()
                    st.markdown("### Result:")
                    st.markdown(st.session_state.last_result)
    
    # Tab 2: Batch Analysis
    with tab2:
        st.subheader("Batch Image Analysis")
        
        uploaded_files = st.file_uploader(
            "Upload multiple images",
            type=["jpg", "jpeg", "png", "webp"],
            accept_multiple_files=True,
            key="batch"
        )
        
        if uploaded_files:
            st.write(f"Uploaded {len(uploaded_files)} images")
            
            batch_analysis = st.selectbox(
                "Analysis Type",
                ["Describe", "Objects", "Text", "Colors", "Tags"]
            )
            
            if st.button("Analyze All", type="primary"):
                model = genai.GenerativeModel('gemini-3-pro')
                
                progress_bar = st.progress(0)
                results = []
                
                for idx, file in enumerate(uploaded_files):
                    image = Image.open(file)
                    
                    prompts = {
                        "Describe": "Briefly describe this image.",
                        "Objects": "List objects in this image.",
                        "Text": "Extract text from this image.",
                        "Colors": "Describe the main colors.",
                        "Tags": "Generate 5 tags for this image."
                    }
                    
                    response = model.generate_content([prompts[batch_analysis], image])
                    results.append((file.name, response.text))
                    
                    progress_bar.progress((idx + 1) / len(uploaded_files))
                
                st.success(f"Analyzed {len(uploaded_files)} images!")
                
                # Display results
                for name, result in results:
                    with st.expander(f"📄 {name}"):
                        st.write(result)
    
    # Tab 3: Compare Images
    with tab3:
        st.subheader("Compare Two Images")
        
        col1, col2 = st.columns(2)
        
        with col1:
            img1 = st.file_uploader("First Image", type=["jpg", "jpeg", "png"], key="img1")
            if img1:
                st.image(Image.open(img1), use_column_width=True)
        
        with col2:
            img2 = st.file_uploader("Second Image", type=["jpg", "jpeg", "png"], key="img2")
            if img2:
                st.image(Image.open(img2), use_column_width=True)
        
        if img1 and img2:
            if st.button("Compare Images", type="primary"):
                with st.spinner("Comparing..."):
                    image1 = Image.open(img1)
                    image2 = Image.open(img2)
                    
                    model = genai.GenerativeModel('gemini-3-pro')
                    
                    # First analysis
                    response1 = model.generate_content(["Describe this image briefly.", image1])
                    response2 = model.generate_content(["Describe this image briefly.", image2])
                    
                    # Comparison
                    comparison_prompt = f"""
                    Compare these two images:
                    Image 1: {response1.text}
                    Image 2: {response2.text}
                    
                    Provide:
                    1. Similarities
                    2. Differences
                    3. Which is better for what purpose
                    """
                    
                    comparison = model.generate_content(comparison_prompt)
                    
                    st.markdown("### Comparison Results:")
                    st.markdown(comparison.text)
else:
    st.warning("Please enter your API key in the sidebar")
'''

with open('image_analysis_advanced.py', 'w') as f:
    f.write(advanced_app_code)

print("Advanced image analysis app saved to image_analysis_advanced.py")
print("\nTo run: streamlit run image_analysis_advanced.py")

## 6. Image Comparison

In [ ]:
class ImageComparer:
    """Compare multiple images."""
    
    def __init__(self):
        self.model = genai.GenerativeModel('gemini-3-pro')
    
    def compare(self, image1: Image.Image, image2: Image.Image) -> dict:
        """Compare two images."""
        # Analyze each image
        desc1 = self.model.generate_content(["Describe this image.", image1])
        desc2 = self.model.generate_content(["Describe this image.", image2])
        
        # Compare
        comparison_prompt = f"""
        Compare these two images:
        
        Image 1: {desc1.text}
        Image 2: {desc2.text}
        
        Provide:
        - Similarities
        - Differences
        - Overall comparison
        """
        
        comparison = self.model.generate_content(comparison_prompt)
        
        return {
            "image1_description": desc1.text,
            "image2_description": desc2.text,
            "comparison": comparison.text
        }

# Test comparison
print("Image comparison example")
print("Create two test images and compare them...")

## 7. Image Quality Assessment

In [ ]:
class ImageQualityAssessor:
    """Assess image quality and provide suggestions."""
    
    def __init__(self):
        self.model = genai.GenerativeModel('gemini-3-pro')
    
    def assess_quality(self, image: Image.Image) -> dict:
        """Assess overall image quality."""
        prompt = """
        Assess this image's quality. Consider:
        - Clarity and sharpness
        - Composition
        - Lighting
        - Colors
        - Overall aesthetic
        
        Provide a rating (1-10) and suggestions for improvement.
        """
        
        response = self.model.generate_content([prompt, image])
        return response.text
    
    def detect_issues(self, image: Image.Image) -> str:
        """Detect potential issues in the image."""
        prompt = """
        Identify any technical or aesthetic issues in this image:
        - Blur or lack of focus
        - Over/underexposure
        - Poor composition
        - Color issues
        - Any other problems
        """
        
        response = self.model.generate_content([prompt, image])
        return response.text

# Test quality assessment
assessor = ImageQualityAssessor()
img = Image.open('test_image.png')

print("Quality Assessment:")
print("=" * 80)
quality = assessor.assess_quality(img)
print(quality)

## 8. Image Classification and Tagging

In [ ]:
class ImageTagger:
    """Generate tags and categories for images."""
    
    def __init__(self):
        self.model = genai.GenerativeModel('gemini-3-pro')
    
    def generate_tags(self, image: Image.Image, n_tags: int = 10) -> list:
        """Generate relevant tags."""
        prompt = f"""
        Generate {n_tags} relevant tags for this image.
        Format as a comma-separated list.
        Tags should be specific, descriptive, and useful for search.
        """
        
        response = self.model.generate_content([prompt, image])
        tags = [tag.strip() for tag in response.text.split(',')]
        return tags
    
    def categorize(self, image: Image.Image) -> dict:
        """Categorize the image."""
        prompt = """
        Categorize this image:
        - Main category (e.g., Nature, People, Architecture, etc.)
        - Sub-category
        - Subject matter
        - Occasion/Context (if applicable)
        
        Format as JSON.
        """
        
        response = self.model.generate_content([prompt, image])
        return response.text
    
    def generate_caption(self, image: Image.Image, style: str = "descriptive") -> str:
        """Generate a caption for the image."""
        style_prompts = {
            "descriptive": "Generate a clear, descriptive caption.",
            "creative": "Generate a creative, engaging caption.",
            "technical": "Generate a technical, detailed caption.",
            "social": "Generate a social media ready caption with emojis."
        }
        
        prompt = f"{style_prompts.get(style, style_prompts['descriptive'])}"
        response = self.model.generate_content([prompt, image])
        return response.text

# Test tagging
tagger = ImageTagger()
img = Image.open('test_image.png')

print("Tags:")
tags = tagger.generate_tags(img, n_tags=5)
print(", ".join(tags))

print("\n" + "="*80 + "\n")

print("Descriptive Caption:")
caption = tagger.generate_caption(img, style="descriptive")
print(caption)

print("\n" + "="*80 + "\n")

print("Creative Caption:")
creative_caption = tagger.generate_caption(img, style="creative")
print(creative_caption)

## 9. Image Search and Similarity

In [ ]:
class ImageSearcher:
    """Search for images based on descriptions."""
    
    def __init__(self):
        self.model = genai.GenerativeModel('gemini-3-pro')
        self.image_database = []
    
    def add_image(self, image: Image.Image, metadata: dict = None):
        """Add an image to the searchable database."""
        # Generate description
        desc = self.model.generate_content(["Describe this image in detail.", image])
        
        self.image_database.append({
            "image": image,
            "description": desc.text,
            "metadata": metadata or {}
        })
    
    def search(self, query: str) -> list:
        """Search for images matching the query."""
        results = []
        
        for idx, item in enumerate(self.image_database):
            # Check if description matches query
            prompt = f"""
            Does this image description match the search query?
            
            Description: {item['description']}
            Query: {query}
            
            Answer with just 'Yes' or 'No' and a relevance score (0-10).
            """
            
            response = self.model.generate_content(prompt)
            
            if "yes" in response.text.lower():
                results.append((idx, item))
        
        return results

print("Image search system example created")
print("Add images to database and search by natural language queries")

## 10. Best Practices for Image Analysis Apps

### Performance Optimization:

1. **Image Resizing**: Resize large images before sending to API
2. **Caching**: Cache analysis results to avoid repeated API calls
3. **Batch Processing**: Process multiple images efficiently
4. **Lazy Loading**: Load images only when needed

### User Experience:

1. **Preview**: Show image previews before analysis
2. **Progress Indicators**: Show progress for batch operations
3. **Error Handling**: Handle unsupported formats gracefully
4. **Export Options**: Allow downloading results

### Technical Considerations:

1. **File Validation**: Validate file types and sizes
2. **Format Support**: Support common image formats (JPEG, PNG, WebP)
3. **Memory Management**: Clean up large images after processing
4. **Rate Limiting**: Implement API rate limiting

### Advanced Features:

1. **Image Enhancement**: Offer preprocessing options
2. **Batch Export**: Export multiple results at once
3. **Comparison Tools**: Compare multiple images
4. **Search Functionality**: Search through analyzed images
5. **Analytics**: Track usage and popular analysis types

### Security:

1. **Input Sanitization**: Validate all inputs
2. **File Size Limits**: Enforce maximum file sizes
3. **Secure Storage**: Don't store sensitive images permanently
4. **Privacy**: Clear uploaded images after processing

## Next Steps

Enhance your image analysis capabilities:
- Add video analysis support
- Implement image generation features
- Build image editing tools
- Create image classification models
- Integrate with cloud storage services

---

## Learn More

Master computer vision and multimodal AI with the **[Gen AI Crash Course](https://www.buildfastwithai.com/genai-course)** by Build Fast with AI!

**Created by [Build Fast with AI](https://www.buildfastwithai.com)**